# Securing an atoti session with Basic authentication


Securing a session comes in two parts:
1. Implementing authentication mechanism to secure access to the session
2. Restricting access of modules or data access by users based on the roles granted

With the [Atoti+ plugin](https://docs.atoti.io/latest/atoti_plus.html), atoti supports multiple [authentication mechanisms](https://docs.atoti.io/latest/lib/atoti/atoti.config.authentication.html) and [basic authentication](https://en.wikipedia.org/wiki/Basic_access_authentication) is the easiest way to set up security.  
This notebook demonstrates the actions required to implement such security:
- Users to be created in atoti. Users will login using their username and password.
- Users to be assigned minimally the role of __ROLE_USER__ to be able to access the atoti web application.

We will explore security with Atoti+ using the [Top 50 Fast Food](https://www.kaggle.com/datasets/stetsondone/top50fastfood) dataset from Kaggle, combined with its parent company information sourced from the internet.


__Note__:  

The notebook is structured in this order:
1. Authentication setup during session instantiation
2. Create BI analytics platform with atoti Community Edition
3. Users and roles management with Atoti+

Mainly, we look at the roles last because we need to know the names of the tables and columns which we want to impose restrictions on.  
Thereafter, we can create the roles with restrictions. Check out [atoti documentation](https://docs.atoti.io) to read more on [securing the session](https://docs.atoti.io/latest/how_tos/security/secure_a_session.html#Configuring-the-authentication-mechanism).  

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=basic-auth" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/discover.png" alt="Try atoti"></a></div>

In [1]:
import atoti as tt
import pandas as pd
from atoti_plus import UserServiceClient

## 1. Authentication setup in atoti

The `realm` below is used to isolate sessions running on the same domain, so if we have only a single session, we can ignore it.  
It will be defaulted to some machine-wide unique ID.

In [2]:
authentication = tt.BasicAuthenticationConfig(realm="atoti Realm")

In [3]:
session = tt.Session(
    port=10011,
    authentication=authentication,
    user_content_storage="./content",
    java_options=["-Dlogging.level.org.springframework.security=DEBUG"],
)

### 1.1 Debug security setup

During the initial setup, it is useful to configure the [Spring Security logging](https://www.baeldung.com/spring-security-enable-logging) to help in debugging any potential issues in the connectivity.  
As shown in the above code snippet, we can turn on logging with `logging.level.org.springframework.security` set to the `DEBUG` level using the `java_options`.

### 1.2 Manage security configuration with UserServiceClient

From [atoti v0.7.2](https://docs.atoti.io/latest/releases/0.7.2.html#added) onwards, atoti uses [UserServiceClient](https://docs.atoti.io/latest/lib/atoti-plus/atoti_plus.user_service_client.user_service_client.html#atoti_plus.UserServiceClient) to manage the dynamic aspects of the security configuration.  
With the client, multiple sessions can be configured with the same security configuration stored on the _user content storage_. The security configuration covers the roles and restrictions.

In [4]:
user_service_client = UserServiceClient.from_session(session)

## 2. Create BI analytics platform with atoti Community Edition

Once the session is created, we can proceed with the usual data loading into atoti table, cube and measures creation.  
Remember to re-execute these cells if you have changed the mode of authentiction.

### 2.1 Table creation

Although we can [`create table`](https://docs.atoti.io/latest/lib/atoti/atoti.session.html#atoti.Session.create_table) before loading data in, we used `read_csv` in our example to create and load data into the atoti tables.

In [5]:
base_tbl = session.read_csv(
    "s3://data.atoti.io/notebooks/security/data/parent_co.csv",
    table_name="parent_co",
    keys=["company", "parent_company"],
    process_quotes=True,
)
base_tbl.head()

,,name
parent_company,company,
Inspire Brands,arbys,Arby's
"Domino's Pizza, Inc.",dominos,Domino's
"Papa Murphy's Holdings, Inc.",papa_murphys,Papa Murphy's
Focus Brands,auntie_annes,Auntie Anne's
Panda Restaurant Group,panda_express,Panda Express


In [6]:
enrichment_tbl = session.read_csv(
    "s3://data.atoti.io/notebooks/security/data/top_50_fast_food_US.csv",
    table_name="top_50",
    keys=["company"],
)
enrichment_tbl.head()

,category,sales_in_millions_2019,sales_per_unit_thousands_2019,franchised_units_2019,company_owned_units_2019,total_units_2019,unit_change_from_2018
company,,,,,,,
popeyes_chicken,chicken,3750,1541,2458,41,2499,131
del_taco,global,850,1554,296,300,596,16
jack_in_the_box,burger,3505,1565,2106,137,2243,6
tim_hortons,sandwich,840,1165,715,0,715,-12
mcdonalds,burger,40413,2912,13154,692,13846,-66


In [7]:
base_tbl.join(enrichment_tbl)

### 2.2. Cube creation

In [8]:
cube = session.create_cube(base_tbl, "Fast food analysis")

### 2.3 Measures creation

In [9]:
h, l, m = cube.hierarchies, cube.levels, cube.measures

In [10]:
m["sales_in_millions"] = tt.agg.sum(enrichment_tbl["sales_in_millions_2019"])
m["sales_per_unit_thousands"] = tt.agg.sum(
    enrichment_tbl["sales_per_unit_thousands_2019"]
)
m["franchised_units"] = tt.agg.sum(enrichment_tbl["franchised_units_2019"])
m["company_owned_units"] = tt.agg.sum(enrichment_tbl["company_owned_units_2019"])
m["total_units"] = tt.agg.sum(enrichment_tbl["total_units_2019"])
m["unit_change_from_2018"] = tt.agg.sum(enrichment_tbl["unit_change_from_2018"])

In [11]:
m["% franchised"] = m["franchised_units"] / m["total_units"]
m["% franchised"].formatter = "DOUBLE[0.00%]"

## 3. Roles management in atoti

### 3.1. Create user

Users' information are retrieved from authentication provider if one is used. However, in the case of basic authentication, users are created in atoti.
Making use of the [BasicSecurity](https://docs.atoti.io/latest/lib/atoti-plus/atoti_plus.security.html#atoti_plus.security.BasicSecurity) module, we create an atoti administrator and a generic atoti user as shown below:

In [12]:
user_service_client.basic.create_user("atoti_admin", password="password")
user_service_client.basic.create_user("atoti_user", password="password")

We cherry-picked two parent companies - Inspire Brands and Restaurant Brands International LLC to demonstrate the roles and access management.  
We will create the users based on the list available on [basic_user_pwd.csv](https://data.atoti.io/notebooks/security/users/basic_user_pwd.csv).

In [13]:
users_df = pd.read_csv("users/basic_user_pwd.csv")

for row in users_df.to_dict(orient="records"):
    if row["Status"] == "Active":
        user = user_service_client.basic.create_user(
            row["User"], password=row["Password"]
        )
        print(
            f"Create user: {user.username} - role(s): {user_service_client.individual_roles[user.username]}"
        )

Create user: Inspire_user1 - role(s): {'ROLE_USER'}
Create user: Inspire_user2 - role(s): {'ROLE_USER'}
Create user: Inspire_manager - role(s): {'ROLE_USER'}
Create user: Restaurant_user1 - role(s): {'ROLE_USER'}
Create user: Restaurant_user2 - role(s): {'ROLE_USER'}
Create user: Restaurant_manager - role(s): {'ROLE_USER'}


The system reserved role `ROLE_USER` is automatically assigned to the created users.  
This meant that these users will all be able to access all the web application and all the available data.  
We can, however, restrict access for the users by assigning them roles with restricted access.

#### 3.1.1 Add/Modify/Delete user with watchdog

We can use third party file watchers such as [watchdog](https://python-watchdog.readthedocs.io/en/stable/) to monitor our flat file containing the list of users and their passwords.  
Along with a status indicator, we can easily add/modify/delete users upon file modification.

In [14]:
from watchdog.events import FileCreatedEvent, FileSystemEventHandler
from watchdog.observers.polling import PollingObserver


class AtotiWatcher(FileSystemEventHandler):
    def on_modified(self, event: FileCreatedEvent):
        try:
            users_df = pd.read_csv(event.src_path)

            for row in users_df.to_dict(orient="records"):
                if (row["User"] not in user_service_client.basic.users) & (
                    row["Status"] == "Active"
                ):
                    user = user_service_client.basic.create_user(
                        row["User"], password=row["Password"]
                    )
                    print(f"Create user: {user} - role(s): {user.roles}")
                else:
                    if row["Status"] == "Inactive":
                        user_service_client.basic.users.pop(row["User"])
                    else:
                        user_service_client.basic.users[row["User"]].password = row[
                            "Password"
                        ]
                    print(f"Update user password: {user}")

        except Exception as error:
            print(error)


observer = PollingObserver()
observer.schedule(AtotiWatcher(), "users")
observer.start()

### 3.2. atoti reserved roles  

The below roles are reserved in atoti and should not be altered by users:
- ROLE_ADMIN: able to access all objects in the web application
- ROLE_USER: able to access all data by default. Access to objects such as dashboards, folders, widgets etc is only upon sharing access granted to role.
- ROLE_SHARE: able to share objects such as dashboards, folders, widgets and filters.  

__All users, including the administrator, require the role *ROLE\_USER* to be able to access the atoti web application.__

Let's assume both users, *atoti\_admin* and *atoti\_user*, have been granted the role __ROLE_USER__ and *atoti\_admin* is also granted the role __ROLE_ADMIN__.  
While both *atoti\_admin* and *atoti\_user* are able to access all data, *atoti\_admin* is able to access all objects such as folders and dashboards.  
*atoti\_user* is able to access only the objects created by the user him/herself. Objects created by other users can only be access upon shared access granted.  

Both *atoti\_admin* and *atoti\_user* will not be able to share objects (via the "Share" icon as shown below) unless granted the role __ROLE_SHARE__.  

<img src="https://data.atoti.io/notebooks/security/img/sharing.png" width="70%"/>

### 3.3. Role creation with restrictions  

Data restriction is based on users' requirement. In our use case, we assumed two groups of users with data access limited to those of their parent company:
- users belonging to parent company _Inspire Brands_
- users belonging to parent company _Restaurant Brands International Inc._

Therefore, we will create two roles to apply the restrictions based on the `parent_company` column from the `parent_co` table.  
We will define key that is a tuple, consisting of the name of the table and its column, along with the restricted values imposed on it. 

__NOTE:__  
- We can skip role creation if there are no restrictions imposed on the role. 
- The value provided under the restrictions is cap-sensitive.

In [15]:
role_inspire = user_service_client.create_role(
    "ATOTI_ROLE_INSPIRE",
    restrictions={("parent_co", "parent_company"): ["Inspire Brands"]},
)

role_restaurant = user_service_client.create_role(
    "ATOTI_ROLE_RESTAURANT",
    restrictions={
        ("parent_co", "parent_company"): ["Restaurant Brands International Inc."]
    },
)

#### 3.3.1 Restricted access from combination of roles

Multiple roles can be assigned to the same user. To demonstrate how the access will change when this happens, we create some other roles that restrict data access by the restaurant category, i.e. column `category` from the table `top_50`.

In [16]:
role_team_burger = user_service_client.create_role(
    "ATOTI_ROLE_BURGER", restrictions={("top_50", "category"): ["burger"]}
)

role_team_sandwich = user_service_client.create_role(
    "ATOTI_ROLE_SANDWICH", restrictions={("top_50", "category"): ["sandwich"]}
)

role_team_snack = user_service_client.create_role(
    "ATOTI_ROLE_SNACK", restrictions={("top_50", "category"): ["snack"]}
)

When combined with the restricted role on the `parent_company`, user's access will be further restricted to based on the restriction of the added role.  

For instance, users who are assigned the role __ATOTI_ROLE_BURGER__ will be able to access all the data under _burger_ category restaurants, regardless of the parent companies.  

However, when the same user is also granted the role __ATOTI_ROLE_INSPIRE__, then the user can only access data of restaurants under parent company _Inspire Brands_ that is of category _burger_. 

### 3.4. Role assignments (Good reference for roles setup in authentication providers)  

We can grant atoti roles directly to users created in atoti without having to perform role mappings like in OIDC.

#### 3.4.1 ROLE_ADMIN, ROLE_USER and ROLE_SHARE

All users created from [BasicSecurity](https://docs.atoti.io/latest/lib/atoti-plus/atoti_plus.security.html#atoti_plus.security.BasicSecurity) are already granted the role __ROLE_USER__ to enable access to the web application.  
Let's assign __ROLE_ADMIN__ to the administrative user, __atoti_admin__ and __ROLE_SHARE__ to the *atoti\_user*.

In [17]:
user_service_client.individual_roles["atoti_admin"].add("ROLE_ADMIN")
user_service_client.individual_roles["atoti_user"].add("ROLE_SHARE")

#### 3.4.2 Multiple roles assignment  

We grant the managers only access to the data available under their parent companies.  
These restrictions will be applied under the role __ATOTI_ROLE_INSPIRE__ and __ATOTI_ROLE_RESTAURANT__ respectively.  

Also, the managers will be granted __ROLE_SHARE__ for them to share the objects such as dashboards and widgets for which they are the owners of.

In [18]:
user_service_client.individual_roles["Inspire_manager"].update(
    [role_inspire.name, "ROLE_SHARE"]
)
user_service_client.individual_roles["Restaurant_manager"].update(
    [role_restaurant.name, "ROLE_SHARE"]
)

Each company has two users that have even more restricted access than the managers.  
User 1 of each company can only access data for restaurants of category _burgers_ with role __ATOTI_ROLE_BURGER__.  
User 2 of each company can only access data for restaurants of category _sandwich_ and _snack_ with roles __ATOTI_ROLE_SANDWICH__ and __ATOTI_ROLE_SNACK__.  

Combined with either the role __ATOTI_ROLE_INSPIRE__ or __ATOTI_ROLE_RESTAURANT__, they will only see the specific category of restaurants under their parent companies.

In [19]:
# access restaurants of category Burger under Inspire Brands
user_service_client.individual_roles["Inspire_user1"].update(
    [role_inspire.name, role_team_burger.name]
)

# access restaurants of category Sandwich or Snack under Inspire Brands
user_service_client.individual_roles["Inspire_user2"].update(
    [role_inspire.name, role_team_sandwich.name, role_team_snack.name]
)

# access restaurants of category Burger under Restaurant Brands International LLC
user_service_client.individual_roles["Restaurant_user1"].update(
    [role_restaurant.name, role_team_burger.name]
)

# access restaurants of category Sandwich or Snack under Restaurant Brands International LLC
user_service_client.individual_roles["Restaurant_user2"].update(
    [role_restaurant.name, role_team_sandwich.name, role_team_snack.name]
)

## 4. Test login and access management in web application

Try out any of these users. Password is simply "password".

___Administrator___
- atoti_admin

___Generic user___
- atoti_user

___Inspire Brands users___
- Inspire_user1
- Inspire_user2
- Inspire_manager

___Restaurant Brands International LLC users___
- Restaurant_user1
- Restaurant_user2
- Restaurant_manager

In [20]:
session.link()

Open the notebook in JupyterLab with the atoti extension enabled to see this link.

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=basic-auth" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/discover-try.png" alt="Try atoti"></a></div>